### APIs

In [ ]:
# pip install requests
!pip install requests
!pip install python-dotenv

### HTTP Methods (CRUD Operations)

In [ ]:

# GET = SELECT (read data)
# Used to retrieve/read data from a server
# Example: Get list of employees

# POST = INSERT (create new data)
# Used to create new records
# Example: Create a new employee record

# PUT = UPDATE (modify existing data)
# Used to update/replace existing records
# Example: Update employee salary

# DELETE = DELETE (remove data)
# Used to remove records
# Example: Delete an employee record

### Status Codes

In [ ]:
# Status codes tell you if your request worked or not
# Like SY-SUBRC in ABAP!

# 2xx = Success (like SY-SUBRC = 0)
200  # OK - Request successful
201  # Created - New record created successfully

# 4xx = Client Error (like SY-SUBRC = 4)
400  # Bad Request - Your request has errors
401  # Unauthorized - You need to login/authenticate
404  # Not Found - The resource doesn't exist

# 5xx = Server Error (like SY-SUBRC = 8)
500  # Internal Server Error - Something broke on the server

# Requests Library (postman counterpart)

### SAP HCM (Mulesoft) via REST API

In [ ]:
import requests
import uuid
from dotenv import load_dotenv
import os
import json

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

url = "https://apim-test.jgsummit.com.ph:8443/qas/api/employee-profile/v2/employee-details" #qas
correlation_id = str(uuid.uuid4())

headers = {        
        "client-id": client_id,
        "client-secret": client_secret,
        "x-correlation-id": correlation_id
        
    }

    # Data payload for the GET request
data = {
        "field": "email-address",        
        "value": "STEPHEN.BONIFACIO@JGSUMMIT.PH",   
        # "value": "raivel.mangilit@jgsummit.com.ph",
        # "value": "JohnPaul.Aprecio2@absi.com.ph",
        # "value": "joyce.cruz@jgsummit.com.ph",
        # "value": "jennifer.rosales@jgsummit.com.ph",
        "integrationType": "outbound",
        "systemId": "JHQ",
        "program": "MIGO_PERSONAL_INFO",      
        "dateFrom": "20250101",        
        "dateTo": "20251231"
    }

response = requests.get(url, params=data, headers=headers)
response_data = response.json()
print(json.dumps(response_data, indent=4))

{
    "responseCode": "408",
    "transactionId": "df93f0a2-359a-44c2-8a8a-7c2c4d4d458b",
    "timestamp": "2025-10-30 T 16:41:28.404Z",
    "errorDetails": {
        "status": "HTTP:TIMEOUT",
        "statusMessage": "HTTP GET on resource 'http://localhost:8903/qas/api/employee-profile/p/v1/perner' failed: Timeout exceeded."
    }
}


### Working with Azure Storage via REST API

In [2]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from io import StringIO

load_dotenv()

url = "https://synapsedl50415.blob.core.windows.net/cit-training/employees.csv"
sas_token = os.getenv("SAS_TOKEN")

full_url = url + "?" + sas_token

response = requests.get(full_url)
    
df = pd.read_csv(StringIO(response.text))
df.head()

,EmployeeID,FirstName,LastName,Department,Position,City,Age,Salary,DateHired
0,E1000,Emma,Gutierrez,IT,Executive,Makati,58,86907,2020-07-08
1,E1001,Maria,Perez,HR,Executive,Taguig,33,88777,2019-10-14
2,E1002,Diego,Santos,Operations,Clerk,Manila,24,43977,2017-12-14
3,E1003,Isabel,Rivera,Finance,Engineer,Taguig,22,73624,2019-04-19
4,E1004,John,Hernandez,Finance,Executive,Makati,54,93321,2014-07-25


### Working with NoSQL (Azure CosmosDB) via REST API

In [3]:
import requests
import hmac
import hashlib
import base64
from datetime import datetime
from urllib.parse import quote
import os
from dotenv import load_dotenv

load_dotenv()

# Configuration
COSMOS_ENDPOINT = "https://jgs-migo-cdb-dev.documents.azure.com:443/"
COSMOSDB_PRIMARY_KEY = os.getenv("COSMOSDB_PRIMARY_KEY")
DATABASE_ID = "migo-chat-history-dev"
CONTAINER_ID = "chat-history"

# util function to generate auth token
def generate_auth_token(verb, resource_type, resource_id, date):
    """Generate authorization token for Cosmos DB"""
    key = base64.b64decode(COSMOSDB_PRIMARY_KEY)
    text = f"{verb.lower()}\n{resource_type.lower()}\n{resource_id}\n{date.lower()}\n\n"
    
    signature = base64.b64encode(
        hmac.new(key, text.encode('utf-8'), hashlib.sha256).digest()
    ).decode()
    
    return quote(f"type=master&ver=1.0&sig={signature}")

In [8]:
query = "SELECT * FROM c WHERE c.type = 'conversation'"

resource_link = f"dbs/{DATABASE_ID}/colls/{CONTAINER_ID}"
date = datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT')

auth_token = generate_auth_token("POST", "docs", resource_link, date)

headers = {
    "Authorization": auth_token,
    "x-ms-date": date,
    "x-ms-version": "2018-12-31",
    "x-ms-documentdb-isquery": "True",
    "x-ms-documentdb-query-enablecrosspartition": "True",
    "Content-Type": "application/query+json"
}

body = {
    "query": query
}

url = f"{COSMOS_ENDPOINT}/{resource_link}/docs"
response = requests.post(url, headers=headers, json=body)

json_res = response.json()

docs = json_res.get('Documents')

docs[:2]


[{'id': 'f34868b5-1896-4e46-a0b4-9c2cda6e8595',
  'app_id': '38475730',
  'userId': 'ciandra.pancho@absi.com.ph',
  'type': 'conversation',
  'title': 'Hi',
  'prompt_tokens': 60028,
  'completion_tokens': 10429,
  'total_tokens': 70457,
  'createdAt': '2024-03-21T07:19:51Z',
  'updatedAt': ['2025-01-03T06:38:30Z'],
  '_rid': 'OrxkAIfzDXkBAAAAAAAAAA==',
  '_self': 'dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzDXkBAAAAAAAAAA==/',
  '_etag': '"000033cc-0000-1800-0000-677785fe0000"',
  '_attachments': 'attachments/',
  '_ts': 1735886334},
 {'id': '1e54b5e8-0ff4-4216-9e9c-39d4fa87ef58',
  'app_id': '38475730',
  'userId': 'ciandra.pancho@absi.com.ph',
  'type': 'conversation',
  'title': 'Hi',
  'prompt_tokens': 167,
  'completion_tokens': 10,
  'total_tokens': 177,
  'createdAt': '2024-03-21T07:21:46Z',
  'updatedAt': '2024-03-21T07:21:48Z',
  '_rid': 'OrxkAIfzDXkGAAAAAAAAAA==',
  '_self': 'dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzDXkGAAAAAAAAAA==/',
  '_etag': '"9a00ec6d-0000-1800-00

In [13]:
conversations_df = pd.DataFrame(docs)
conversations_df.sample(5)

,id,app_id,userId,type,title,prompt_tokens,completion_tokens,total_tokens,createdAt,updatedAt,_rid,_self,_etag,_attachments,_ts,category
520,d557cce7-b700-46d1-b58a-e9b283a7dc7d,38475730,ciandra.pancho@absi.com.ph,conversation,hi,0,0,0,2024-04-11T22:36:21Z,2024-04-11T22:36:21Z,OrxkAIfzDXk-CAAAAAAAAA==,dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzD...,"""9a00a976-0000-1800-0000-6667c6540000""",attachments/,1718077012,None
773,7f4f279f-2049-4279-8c8f-709db4ab8693,38475730,ej.dizon@jgsummit.com.ph,conversation,How to file SSS materity Leave,3328,955,4283,2024-04-26T01:04:45Z,2024-04-26T01:11:34Z,OrxkAIfzDXmtDgAAAAAAAA==,dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzD...,"""9a00817d-0000-1800-0000-6667c65b0000""",attachments/,1718077019,None
174,b5a21cec-d4a7-4aad-aa02-91d16da44c86,38475730,ciandra.pancho@absi.com.ph,conversation,Ano ang sss loan?,1941,588,2529,2024-04-05T01:23:48Z,2024-04-05T01:24:03Z,OrxkAIfzDXmNBAAAAAAAAA==,dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzD...,"""9a00bf72-0000-1800-0000-6667c6500000""",attachments/,1718077008,None
226,3e6b2095-4e90-4648-8bf8-ed07903c1a4d,38475730,ciandra.pancho@absi.com.ph,conversation,hi,0,0,0,2024-04-11T02:46:34Z,2024-04-11T02:46:34Z,OrxkAIfzDXmBBQAAAAAAAA==,dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzD...,"""9a00c173-0000-1800-0000-6667c6510000""",attachments/,1718077009,None
209,b39f27bb-bb2f-4963-b3ef-51f42aa99459,38475730,ciandra.pancho@absi.com.ph,conversation,Ano ang sss loan?,973,295,1268,2024-04-08T01:50:36Z,2024-04-08T01:50:37Z,OrxkAIfzDXk+BQAAAAAAAA==,dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzD...,"""9a007773-0000-1800-0000-6667c6510000""",attachments/,1718077009,None


# SDKs

### Azure Storage

In [ ]:
# install azure storage blob sdk
!pip install azure-storage-blob

In [18]:
from azure.storage.blob import BlobServiceClient
import os
from dotenv import load_dotenv
import pandas as pd
from io import StringIO

load_dotenv()

# Configuration
sas_token = os.getenv("SAS_TOKEN")

# Initialize BlobServiceClient with SAS token
blob_service_client = BlobServiceClient(
    account_url="https://synapsedl50415.blob.core.windows.net",
    credential=sas_token
)

# Get blob client
blob_client = blob_service_client.get_blob_client(
    container="cit-training",
    blob="employees.csv"
)

# Download blob content
blob_data = blob_client.download_blob()
csv_content = blob_data.readall().decode('utf-8')

# Read into DataFrame
df = pd.read_csv(StringIO(csv_content))
df.head()

,EmployeeID,FirstName,LastName,Department,Position,City,Age,Salary,DateHired
0,E1000,Emma,Gutierrez,IT,Executive,Makati,58,86907,2020-07-08
1,E1001,Maria,Perez,HR,Executive,Taguig,33,88777,2019-10-14
2,E1002,Diego,Santos,Operations,Clerk,Manila,24,43977,2017-12-14
3,E1003,Isabel,Rivera,Finance,Engineer,Taguig,22,73624,2019-04-19
4,E1004,John,Hernandez,Finance,Executive,Makati,54,93321,2014-07-25


### Azure CosmosDB

In [ ]:
# Access CosmosDB via SDK
!pip install azure-cosmos

In [ ]:
from azure.cosmos import CosmosClient
import os
from dotenv import load_dotenv

load_dotenv()

# Configuration
COSMOS_ENDPOINT = "https://jgs-migo-cdb-dev.documents.azure.com:443/"
COSMOSDB_PRIMARY_KEY = os.getenv("COSMOSDB_PRIMARY_KEY")
DATABASE_ID = "migo-chat-history-dev"
CONTAINER_ID = "chat-history"

# Initialize Cosmos Client
client = CosmosClient(COSMOS_ENDPOINT, COSMOSDB_PRIMARY_KEY)

# Get database and container
database = client.get_database_client(DATABASE_ID)
container = database.get_container_client(CONTAINER_ID)

# Query using SDK
query = "SELECT * FROM c WHERE c.type = 'conversation'"

items = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

# Display first 2 items
items[:2]

[{'id': 'f34868b5-1896-4e46-a0b4-9c2cda6e8595',
  'app_id': '38475730',
  'userId': 'ciandra.pancho@absi.com.ph',
  'type': 'conversation',
  'title': 'Hi',
  'prompt_tokens': 60028,
  'completion_tokens': 10429,
  'total_tokens': 70457,
  'createdAt': '2024-03-21T07:19:51Z',
  'updatedAt': ['2025-01-03T06:38:30Z'],
  '_rid': 'OrxkAIfzDXkBAAAAAAAAAA==',
  '_self': 'dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzDXkBAAAAAAAAAA==/',
  '_etag': '"000033cc-0000-1800-0000-677785fe0000"',
  '_attachments': 'attachments/',
  '_ts': 1735886334},
 {'id': '1e54b5e8-0ff4-4216-9e9c-39d4fa87ef58',
  'app_id': '38475730',
  'userId': 'ciandra.pancho@absi.com.ph',
  'type': 'conversation',
  'title': 'Hi',
  'prompt_tokens': 167,
  'completion_tokens': 10,
  'total_tokens': 177,
  'createdAt': '2024-03-21T07:21:46Z',
  'updatedAt': '2024-03-21T07:21:48Z',
  '_rid': 'OrxkAIfzDXkGAAAAAAAAAA==',
  '_self': 'dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzDXkGAAAAAAAAAA==/',
  '_etag': '"9a00ec6d-0000-1800-00

In [ ]:
# Convert to DataFrame
conversations_df = pd.DataFrame(items)
conversations_df.sample(5)

### Azure SQL

In [ ]:
# install pyodbc
!pip install pyodbc

In [ ]:
# install odbc driver for sql server via powershell in admin mode.
!choco install sqlserver-odbcdriver -y

In [ ]:
import pyodbc

USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

conn_str = (
    "Driver={ODBC Driver 18 for SQL Server};"
    "Server=tcp:sqlserver50415.database.windows.net,1433;"
    "Database=timekeeping;"
    f"UID={USERNAME};"
    f"PWD={PASSWORD};"
    "Encrypt=yes;"
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
query = "SELECT TOP 10 * FROM [dbo].[time_data]"

cursor.execute(query)
rows = cursor.fetchall()

# display rows
for row in rows:
    print(row)

# close connection
conn.close()

('20000004', 'Juan dela Cruz', datetime.date(2023, 1, 6), 'OT Special Holiday', 4, Decimal('600.00'), 'Stephen Bonifacio')
('20000004', 'Juan dela Cruz', datetime.date(2023, 1, 6), 'LWOP', 8, Decimal('-1200.00'), 'Stephen Bonifacio')
('20000004', 'Juan dela Cruz', datetime.date(2023, 1, 6), 'Tardiness', 0, Decimal('-39.00'), 'Stephen Bonifacio')
('20000005', 'Maria Cruz', datetime.date(2023, 1, 13), 'OT SH Rest Day', 2, Decimal('300.00'), 'Stephen Bonifacio')
('20000005', 'Maria Cruz', datetime.date(2023, 1, 13), 'Tardiness', 0, Decimal('-73.50'), 'Stephen Bonifacio')
('20000004', 'Juan dela Cruz', datetime.date(2023, 1, 14), 'OT Special Holiday', 3, Decimal('450.00'), 'Stephen Bonifacio')
('20000004', 'Juan dela Cruz', datetime.date(2023, 1, 14), 'Tardiness', 0, Decimal('-121.50'), 'Stephen Bonifacio')
('20000002', 'Carlos Bautista', datetime.date(2023, 1, 28), 'Shift Differential', 4, Decimal('600.00'), 'Stephen Bonifacio')
('20000002', 'Carlos Bautista', datetime.date(2023, 1, 28), 

### Load as pandas df

In [ ]:
# open connection
conn = pyodbc.connect(conn_str)

# run query
sql_df = pd.read_sql_query(query, conn)

# close connection
conn.close()

# display df
sql_df.head()

C:\Users\sb11343\AppData\Local\Temp\ipykernel_13932\2890645671.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_df = pd.read_sql_query(query, conn)


,employee_number,employee_name,date,time_type,hours,amount_php,supervisor_name
0,20000004,Juan dela Cruz,2023-01-06,OT Special Holiday,4,600.0,Stephen Bonifacio
1,20000004,Juan dela Cruz,2023-01-06,LWOP,8,-1200.0,Stephen Bonifacio
2,20000004,Juan dela Cruz,2023-01-06,Tardiness,0,-39.0,Stephen Bonifacio
3,20000005,Maria Cruz,2023-01-13,OT SH Rest Day,2,300.0,Stephen Bonifacio
4,20000005,Maria Cruz,2023-01-13,Tardiness,0,-73.5,Stephen Bonifacio


## Exercise

### Employee Data Pipeline (ETL)

**Objective**: Build an ETL pipeline that extracts employee data from SAP HCM API and loads it to Azure Blob Storage as a CSV file.

**Scenario**: Your team needs to extract employee information for a list of employees from the SAP HCM system and store it in Azure Storage for reporting purposes.

**Requirements**:
1. **Extract**: Fetch employee data from SAP HCM API for the following employees:
   - STEPHEN.BONIFACIO@JGSUMMIT.PH
   - raivel.mangilit@jgsummit.com.ph
   - JohnPaul.Aprecio2@absi.com.ph
   - joyce.cruz@jgsummit.com.ph   

2. **Transform**: Parse the API response and create a clean dataset with the following column names:
   - Email Address (from request)
   - Permanent Address (Full)   
   - Current Address (Full)
   - TIN (Tax Identification Number)
   - SSS Number
   - Pag-IBIG Number
   - PhilHealth Number

3. **Load**: Save the transformed data as a CSV file to Azure Blob Storage in the `cit-training` container with filename: `employee_data_YYYYMMDD_yourname.csv` (use current date)s

**Hints**:
- Handle API errors gracefully (some employees might not exist)
- Use pandas DataFrame for transformation
- Set values of non-existent fields as 'Not Available'.